### 1. ID3: Iterative Dichotomiser

- 엔트로피: -np.sum(p(x) * np.log(p(x)))
- Information Gain: 정보 이득은 부모 노드의 엔트로피 H(t)와 자식 노드 엔트로피의 가중 평균의 차이다.
    - 정보 이득을 최대화하는 것으로 분기한다.
- C4.5: 연속형 변수에 사용할 수 있고 가지치기가 가능하다. (인스턴스가 거의 없는 잎 노드를 없앤다)

### 2. Gini Impurity

- 지니 계수: 1 - np.sum(np.square(p(i|t)))
     - 지니 계수도 모든 데이터가 동일하게 나뉠 수록 가장 높은 값을 가진다.
     - 실제로 Information Gain을 쓰나, 지니 불순도를 쓰나 성능에 큰 차이가 없다.
     

In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

In [4]:
df = pd.read_csv('./ad-dataset/ad.data', header=None)

/Users/sailyourlife/anaconda3/envs/nextopt/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
explantory_variable_columns = set(df.columns.values)
explantory_variable_columns.remove(len(df.columns.values)-1)

# final Column
response_varible_column = df[len(df.columns.values) - 1]

### regex

In [18]:
y = [1 if e == 'ad.' else 0 for e in response_varible_column]
X = df[list(explantory_variable_columns)].copy()
X.replace(to_replace=' *?', value=-1, regex=True, inplace=True)

### train test split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [22]:
pipeline = Pipeline([
    ('clf', DecisionTreeClassifier(criterion='entropy'))
])

parameters = {
    'clf__max_depth': (150, 155, 160),
    'clf__min_samples_split': (2, 3),
    'clf__min_samples_leaf': (1,2,3)
}

In [24]:
if __name__ == "__main__":
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, scoring='f1')
    grid_search.fit(X_train, y_train)
    best_parameters= grid_search.best_estimator_.get_params()
    print("Best sore: %0.3f" % grid_search.best_score_)
    print("Best paramters set:")
    
    for param_name in sorted(parameters.keys()):
        print('t%s: %r'% (param_name, best_parameters[param_name]))
        
    predictions = grid_search.predict(X_test)
    print(classification_report(y_test, predictions))

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:    9.7s finished


Best sore: 0.885
Best paramters set:
tclf__max_depth: 160
tclf__min_samples_leaf: 3
tclf__min_samples_split: 3
             precision    recall  f1-score   support

          0       0.97      0.99      0.98       707
          1       0.92      0.78      0.84       113

avg / total       0.96      0.96      0.96       820



- Decision Tree는 Greedy 방식이다.
    - 지역 최적화를 통해 효율성을 학습한다. 
    

In [ ]:
2